In [ ]:
import pandas as pd 
import numpy as np
import datetime
import os 
os.chdir("/home/luisfernando/Git Code/Introduction-to-Data-Analysis/Lecture 3")
df = pd.read_excel("random_time_data.xlsx")
df

In [ ]:
df.dtypes

In [ ]:
pd.Timestamp(year=2020, month=1, day=1, hour=0, minute=0, second=0)

In [ ]:
pd.Timestamp(year=2020, month=1, day=1, hour=0, minute=0)

In [ ]:
pd.Timestamp(year=2020, month=1, day=1, hour=0)

In [ ]:
pd.Timestamp(year=2020, month=1, day=1)

In [ ]:
pd.Timestamp(year=2018, month=3, day=4, hour=3, minute=56, second=14)

In [ ]:
pd.Timestamp(year=2018, month=3, day=4, hour=3, minute=45)

In [ ]:
df.Timestamp

In [ ]:
df.Timestamp.dt.time

In [ ]:
df.Timestamp.dt.date

In [ ]:
df.Timestamp.between(pd.Timestamp(year=2020, month=1, day=1, hour=0, minute=0), pd.Timestamp(year=2020, month=2, day=1, hour=0, minute=0))

In [ ]:
df[df.Timestamp.between(pd.Timestamp(year=2020, month=1, day=1, hour=0, minute=0), pd.Timestamp(year=2020, month=2, day=1, hour=0, minute=0))]

In [ ]:
df[df.Timestamp.between(pd.Timestamp(year=2020, month=1, day=1, hour=0, minute=0), pd.Timestamp(year=2020, month=2, day=1, hour=0, minute=0),  inclusive = False)]

In [ ]:
january = df[df.Timestamp.between(pd.Timestamp(year=2020, month=1, day=1, hour=0, minute=0), pd.Timestamp(year=2020, month=2, day=1, hour=0, minute=0))]
january

In [ ]:
january = january.reset_index(drop=True)
january

In [ ]:
january.Demand.max()

In [ ]:
january.Demand[df.Peak == "Mid-Peak"]

In [ ]:
january.Demand[df.Peak == "Mid-Peak"].max()

In [ ]:
billing = pd.read_excel("billing_month.xlsx")
billing

In [ ]:
billing.dtypes

In [ ]:
billing.billing_month

In [ ]:
billing.billing_month[0]

In [ ]:
billMonth = billing.billing_month
offset = pd.DateOffset(minutes=15)
month = {}
monthLength = len(billMonth) - 1
for i in range (0,monthLength,1):
    end = billMonth[i + 1] - offset
    month[i] = df[df.Timestamp.between(billMonth[i], end)]
month

In [ ]:
month[0]

In [ ]:
np.nan

In [ ]:
example = pd.DataFrame(np.zeros([10, 4])*np.nan, columns = ["One", "Two", "Three", "Four"])
example

In [ ]:
def energy(df, month):
    df = df.reset_index(drop=True)

    if len(month) != 0:
        monthLength = len(month) 
    else:
         monthLength = 0
            
    monthTitle = {}
    for i in range(0, monthLength, 1): # iterates through every billing month df
        monthTitle[i] = month[i].Timestamp.min().strftime('%b_%d_%Y') + ' to ' + month[i].Timestamp.max().strftime('%b_%d_%Y')

    columns = ['Billing Month', 'kiloWattHour Total', 'kiloWattHour Super Off-Peak', 'kiloWattHour Off-Peak', 'kiloWattHour Mid-Peak', 'kiloWattHour On-Peak']
    col_len = int(len(columns))
    kiloWattHour = pd.DataFrame(np.zeros([monthLength, col_len])*np.nan, columns = columns)
    for i in range(0, monthLength, 1):
        month[i] = month[i].reset_index(drop=True)
        kiloWattHour['Billing Month'][i] = monthTitle[i]
        kiloWattHour['kiloWattHour Total'][i] = month[i].Demand.sum() / 4
        kiloWattHour['kiloWattHour Super Off-Peak'][i] = month[i].Demand[month[i].Peak == 'Super Off-Peak'].sum() / 4
        kiloWattHour['kiloWattHour Off-Peak'][i] = month[i].Demand[month[i].Peak == 'Off-Peak'].sum() / 4
        kiloWattHour['kiloWattHour Mid-Peak'][i] = month[i].Demand[month[i].Peak == 'Mid-Peak'].sum() / 4
        kiloWattHour['kiloWattHour On-Peak'][i] = month[i].Demand[month[i].Peak == 'On-Peak'].sum() / 4

    kiloWattHour.loc[len(kiloWattHour)] = np.nan
    kiloWattHour = kiloWattHour.shift()
    kiloWattHour['Billing Month'][0] = 'Entire Interval'
    kiloWattHour['kiloWattHour Total'][0] = df.Demand.sum() / 4
    kiloWattHour['kiloWattHour Super Off-Peak'][0] = df.Demand[df.Peak == 'Super Off-Peak'].sum() / 4
    kiloWattHour['kiloWattHour Off-Peak'][0] = df.Demand[df.Peak == 'Off-Peak'].sum() / 4
    kiloWattHour['kiloWattHour Mid-Peak'][0] = df.Demand[df.Peak == 'Mid-Peak'].sum() / 4
    kiloWattHour['kiloWattHour On-Peak'][0] = df.Demand[df.Peak == 'On-Peak'].sum() / 4

    return kiloWattHour 

In [ ]:
kiloWattHour = energy(df,month)
kiloWattHour

In [ ]:
def max_peak(df, month):
    df = df.reset_index(drop=True)

    if len(month) != 0:
        monthLength = len(month) 
    else:
         monthLength = 0

    monthTitle = {}
    for i in range(0, monthLength, 1): # iterates through every billing month df
        monthTitle[i] = month[i].Timestamp.min().strftime('%b_%d_%Y') + ' to ' + month[i].Timestamp.max().strftime('%b_%d_%Y')

    columns = ['Billing Month', 'max_peak Total', 'max_peak Super Off-Peak', 'max_peak Off-Peak', 'max_peak Mid-Peak', 'max_peak On-Peak']
    col_len = int(len(columns))
    max_peak = pd.DataFrame(np.zeros([monthLength, col_len])*np.nan, columns = columns)
    for i in range(0, monthLength, 1):
        month[i] = month[i].reset_index(drop=True)
        max_peak['Billing Month'][i] = monthTitle[i]
        max_peak['max_peak Total'][i] = month[i].Demand.max()
        max_peak['max_peak Super Off-Peak'][i] = month[i].Demand[month[i].Peak == 'Super Off-Peak'].max()
        max_peak['max_peak Off-Peak'][i] = month[i].Demand[month[i].Peak == 'Off-Peak'].max()
        max_peak['max_peak Mid-Peak'][i] = month[i].Demand[month[i].Peak == 'Mid-Peak'].max()
        max_peak['max_peak On-Peak'][i] = month[i].Demand[month[i].Peak == 'On-Peak'].max()

    max_peak.loc[len(max_peak)] = np.nan
    max_peak = max_peak.shift()
    max_peak['Billing Month'][0] = 'Entire Interval'
    max_peak['max_peak Total'][0] = df.Demand.max()
    max_peak['max_peak Super Off-Peak'][0] = df.Demand[df.Peak == 'Super Off-Peak'].max()
    max_peak['max_peak Off-Peak'][0] = df.Demand[df.Peak == 'Off-Peak'].max()
    max_peak['max_peak Mid-Peak'][0] = df.Demand[df.Peak == 'Mid-Peak'].max()
    max_peak['max_peak On-Peak'][0] = df.Demand[df.Peak == 'On-Peak'].max()

    return max_peak

In [ ]:
max_peak = max_peak(df, month)
max_peak

In [ ]:
def energy_percentage(df, month):
    df = df.reset_index(drop=True)

    if len(month) != 0:
        monthLength = len(month) 
    else:
         monthLength = 0
         
    monthTitle = {}
    for i in range(0, monthLength, 1): # iterates through every billing month df
        monthTitle[i] = month[i].Timestamp.min().strftime('%b_%d_%Y') + ' to ' + month[i].Timestamp.max().strftime('%b_%d_%Y')

    columns = ['Billing Month', 'Energy_Percentage Total', 'Energy_Percentage Super Off-Peak', 'Energy_Percentage Off-Peak', 'Energy_Percentage Mid-Peak', 'Energy_Percentage On-Peak']
    col_len = int(len(columns))
    Energy_Percentage = pd.DataFrame(np.zeros([monthLength, col_len])*np.nan, columns = columns)
    for i in range(0, monthLength, 1):
        month[i] = month[i].reset_index(drop=True)
        Energy_Percentage['Billing Month'][i] = monthTitle[i]
        month_sum = month[i].Demand.sum()
        Energy_Percentage['Energy_Percentage Super Off-Peak'][i] = month[i].Demand[month[i].Peak == 'Super Off-Peak'].sum() / month_sum
        Energy_Percentage['Energy_Percentage Off-Peak'][i] = month[i].Demand[month[i].Peak == 'Off-Peak'].sum() / month_sum
        Energy_Percentage['Energy_Percentage Mid-Peak'][i] = month[i].Demand[month[i].Peak == 'Mid-Peak'].sum() / month_sum
        Energy_Percentage['Energy_Percentage On-Peak'][i] = month[i].Demand[month[i].Peak == 'On-Peak'].sum() / month_sum
        Energy_Percentage['Energy_Percentage Total'][i] = Energy_Percentage['Energy_Percentage Super Off-Peak'][i] + Energy_Percentage['Energy_Percentage Off-Peak'][i] + Energy_Percentage['Energy_Percentage Mid-Peak'][i] + Energy_Percentage['Energy_Percentage On-Peak'][i]

    Energy_Percentage.loc[len(Energy_Percentage)] = np.nan
    Energy_Percentage = Energy_Percentage.shift()
    Energy_Percentage['Billing Month'][0] = 'Entire Interval'
    df_sum = df.Demand.sum()
    Energy_Percentage['Energy_Percentage Super Off-Peak'][0] = df.Demand[df.Peak == 'Super Off-Peak'].sum() / df_sum
    Energy_Percentage['Energy_Percentage Off-Peak'][0] = df.Demand[df.Peak == 'Off-Peak'].sum() / df_sum
    Energy_Percentage['Energy_Percentage Mid-Peak'][0] = df.Demand[df.Peak == 'Mid-Peak'].sum() / df_sum
    Energy_Percentage['Energy_Percentage On-Peak'][0] = df.Demand[df.Peak == 'On-Peak'].sum() / df_sum
    Energy_Percentage['Energy_Percentage Total'][0] = Energy_Percentage['Energy_Percentage Super Off-Peak'][0] + Energy_Percentage['Energy_Percentage Off-Peak'][0] + Energy_Percentage['Energy_Percentage Mid-Peak'][0] + Energy_Percentage['Energy_Percentage On-Peak'][0]

    return Energy_Percentage

In [ ]:
kW_percentage = energy_percentage(df, month)
kW_percentage